In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/transformer_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/transformer_export/1575453050/variables/variables


In [6]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  #print('Shape:', predictions['output'].shape)
  max_len = 0
  output = None
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    sent = ' '.join(indices)
    if len(indices) > max_len:
      max_len = len(indices)
      output = sent
    #print('Rank_{}:'.format(j+1), sent)
  print('Output:', output)
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:你好
Output: 你 好 啊
Cost 2.21 sec

Input:早上好
Output: 早 上 好 ！
Cost 0.21 sec

Input:晚上好
Output: 主 人 晚 上 好 O ( ∩ _ ∩ ) O
Cost 0.32 sec

Input:再见
Output: 再 见 ！ ！
Cost 0.15 sec

Input:好久不见
Output: 好 久 不 见 [ 嘻 嘻 ]
Cost 0.23 sec

Input:想死你了
Output: 主 人 我 也 想 死 你 了
Cost 0.18 sec

Input:谢谢你
Output: 谢 谢 你
Cost 0.15 sec

Input:爱你
Output: 我 也 爱 你   主 人 〜 ( ^ ○ ^ )
Cost 0.26 sec

Input:哈啰
Output: 哈 哈 哈 哈
Cost 0.20 sec

Input:你叫什么名字
Output: 你 叫 什 么 我 就 叫 什 么 名 字
Cost 0.33 sec

Input:你几岁了
Output: 我 今 天 就 几 岁 了
Cost 0.18 sec

Input:现在几点了
Output: 现 在 几 点 了
Cost 0.18 sec

Input:今天天气怎么样
Output: 今 天 怎 么 了
Cost 0.15 sec

Input:我们现在在哪里
Output: 在 哪 里 我 就 在 哪 里 呦 ！
Cost 0.20 sec

Input:你能给我讲个笑话吗
Output: 我 喜 欢 给 你 一 个
Cost 0.28 sec

Input:你是男孩还是女孩呀
Output: 我 是 小 通
Cost 0.27 sec

Input:你会几种语言呀
Output: 全 世 界 界 语 言 的 我 都 会
Cost 0.20 sec

Input:你能陪我玩吗
Output: 玩 你 妹 啊
Cost 0.16 sec

Input:说话可以大声一点吗
Output: 可 以 。
Cost 0.19 sec

Input:天气真好
Output: 天 气 很 好
Cost 0.17 sec

Input:天气太糟糕了
Output: 天 气 糟 糕 了 ?
Cost 0.25 s

KeyboardInterrupt: ignored